Basic Imports and Device Setup

In [ ]:
# Standard Libraries
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Scikit-learn for Preprocessing and Metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.manifold import TSNE
from scipy.stats import gaussian_kde

In [ ]:
import os
import requests
import pickle